## Big(ger) Data Fundamentals with Pandas
This shares a "Big(ger) Data Fundamentals example in Python/Pandas.  This example mirrors JMP exercises with the same sample sales data, but Pandas provides a lot of flexibility to work with even bigger data and to "script" the analyses for repeated use.</br></br>JDL / Data Delve LLC / http://datadelveengineer.com / 12/9/22

### Jupyter notebook hints for running these scripts
* Install Anaconda Individual Edition for either Windows or Mac.  This gets up-to-date versions of Python and Pandas on your computer
* Launch Anaconda and open the Jupyter Lab tool there
* In Jupyter Lab (opens in your browser but as a purely local/non-internet tool), open this notebook
* Choose the Run menu / Run All Cells to run the examples here

### Pandas basics
Pandas is widely used.  It was developed in the early to middle 2010's with an initial focus on the financial industry.  Its core concept for holding data is the "DataFrame."  A DataFrame is synonomous with "data table with a bunch of metadata about the table and its columns." Pandas is ingeniously coded to be efficient with  data sets greater than 1 MM rows.

The first statements below simply import the Pandas and Numpy Python libraries. For code readability, these statements should be at the top of a Jupyter notebook like this one. The "as pd" allow referring to Pandas commands as `pd.method(options)` throughout the rest of the notebook's script statements.  The 'pd' and 'np' abbreviations can actually be any text strings, but you should use 'pd' and 'np' to avoid confusion

In [1]:
import pandas as pd
import numpy as np

### A first DataFrame example
We will shortly read data into a DataFrame from a separate Excel file. To get started though, here is a quick example that creates a DataFrame from Python "lists" --one for each column.  There are three columns and three rows of data.  The second line is what does the "printing out" in this case.</br></br>Note the automatically-generated index column at the left.  This lets us refer to an individual row by index.

In [3]:
df_little = pd.DataFrame(data={'one_column':['a', 'b', 'c'], 'second_column':[10,15, 20], 'third_column':[100, 150, 200]})
df_little

,one_column,second_column,third_column
0,a,10,100
1,b,15,150
2,c,20,200


by default, this DataFrame has what's called a Range Index consisting of zero-based integers 0, 1 and 2. However, one or more of the columns can be used as indices too.  This enables slicing out rows by index values as names.  Here is an example of using the first data column as the index. Notice how the index name is offset below the column names to indicate that `one_column` is the index

In [5]:
df_little = df_little.set_index('one_column')
df_little

,second_column,third_column
one_column,,
a,10,100
b,15,150
c,20,200


The `.iloc` and `.loc` functions are useful for accessing or "slicing" out rows.  `.iloc` allows referring to rows and columns by zero-based integer aka row number. `.loc` accesses rows by index and column name.  Below are examples of getting individual values with either method. A couple of things to notice:
* The print() statement is useful to print out multiple items from a Jupyter Notebook cell --don't forget the parentheses!
* Since the row and column indices are zero-based, `[1, 1]` refers to the second row and second, non-index (aka data) column
* Notice that `.iloc` and `.loc` use square brackets to enclose their arguments

In [9]:
print(df_little.iloc[1, 1])
print(df_little.loc['b', 'third_column'])

150
150


### Working with the sample Sales Data
The Bigger Data Fundamentals sample data can be imported into a DataFrame --directly from the Excel file. The stat at the table bottom says it all:  Just like that we have all 24,546 rows in a DataFrame.  With an even bigger data set, we would likely shift to reading the data in CSV format to avoid hitting Excel's 1 MM row limit.  That would use the more common (and much faster) `pd.read_csv()` method.

In [11]:
dfSales = pd.read_excel('Example_Sales_Data.xlsx', sheet_name='Sheet1')
dfSales

,Quarter,Channel,Customer ID,Country,Product Line,Revenue,Product Cost,Customer Service Cost,Profit
0,Q3,ATM,FRT,USA,Credit Products,6044,3998,413,1633
1,Q1,ATM,MRT,USA,Credit Products,4686,3229,643,815
2,Q4,ATM,PBI,USA,Deposit Products,6063,7440,1842,-3219
3,Q1,ATM,PBI,USA,Deposit Products,4682,6127,1118,-2563
4,Q4,ATM,MRT,USA,Deposit Products,6320,7913,1854,-3447
...,...,...,...,...,...,...,...,...,...
24541,Q4,ATM,WEB,USA,Credit Products,4593,5302,1078,-1788
24542,Q1,ATM,PBI,USA,Revolving Credit Products,4268,1638,363,2267
24543,Q3,ATM,MRT,USA,Deposit Products,5505,3623,875,1007
24544,Q1,ATM,MAM,USA,Credit Products,5449,3911,561,977


In [14]:
print(dfSales.columns)

Index(['Quarter', 'Channel', 'Customer ID', 'Country', 'Product Line',
       'Revenue', 'Product Cost', 'Customer Service Cost', 'Profit'],
      dtype='object')


In [15]:
print(dfSales.index)

RangeIndex(start=0, stop=24546, step=1)


We won't go into it here, but there are other, even more efficient data formats for larger datasets.  Feather format (created as part of the PyArrow open-source project) is an example, and there are Pandas methods for reading and writing it as well.  This can dramatically reduce file size.  On the file size scale, Excel (the "fat one") >> CSV (on a diet but needs to keep pedaling) >> Feather ("lean and mean").

### Creating a Subset of the Sample Data
Now we are ready to get to work.  We can subset the DataFrame using a notation that puts a true/false "Boolean" expression in brackets after the name of the master DataFrame.
* We can create a subset DataFrame with the general statement like: `dfSubset = dfMaster[<<T/F expression about each row>>]
* A lot of the T/F expression are things interpreted like "rows where column X = "XYZ".  These use a double equals sign to denote the comparison (as opposed to using an equals sign to set a variable's value) so `df['Country'] == "Canada"` evaluates to True or False based on whether the Country column has the value "Canada"

Here is an example. a T/F expression like this that is known as a "mask" in Pandas, so we use that as a variable name to be clear.  Masks are used to efficiently tell Pandas what rows to keep in a subset.  The variable called "mask" actually refers to a series of 24,546 True or False values.  This can be overlaid on our DataFrame to subset the DataFrame.  One of the crazy ingenious things about Pandas is that this actually has tremendously fast performance with large tables

In [16]:
mask = dfSales['Country'] == 'Canada'
mask

0        False
1        False
2        False
3        False
4        False
         ...  
24541    False
24542    False
24543    False
24544    False
24545    False
Name: Country, Length: 24546, dtype: bool

Here is how to show just the rows that meet the mask criteria --1388 rows for "Canada"

In [17]:
dfSales[mask]

,Quarter,Channel,Customer ID,Country,Product Line,Revenue,Product Cost,Customer Service Cost,Profit
87,Q2,ATM,EBP,Canada,Deposit Products,762,932,136,-305
160,Q2,ATM,AGR,Canada,Revolving Credit Products,563,635,65,-137
188,Q1,EML,EBP,Canada,Credit Products,378,226,42,110
211,Q1,ATM,AGR,Canada,Revolving Credit Products,516,360,37,119
214,Q3,ATM,MRT,Canada,Credit Products,314,222,50,42
...,...,...,...,...,...,...,...,...,...
24252,Q4,ATM,EBP,Canada,Revolving Credit Products,1251,948,236,67
24274,Q3,BRH,EBP,Canada,Deposit Products,547,706,78,-237
24294,Q2,ATM,EBP,Canada,Revolving Credit Products,886,770,137,-22
24331,Q3,ATM,EBP,Canada,Deposit Products,800,809,137,-146


</br></br>If we need to work further with the subset, it makes sense to assign it to a new name.  Let's do this for Country = "USA".  We can re-use the "mask" name for this.  It will work for Canada above, but the name gets reassigned in the following statement based on the top-to-bottom flow of running the cells.  `df.index.size` is a good way to just print out the number of rows in a DataFrame, which is 3074 for "USA".  The following commands show examples of printing the "Info" (how many rows are populated, what data types) and "descibing" a DataFrame to get summary info. </br></br> Try adding a cell with `dfSales_USA.to_excel('TestFile.xlsx', index=False)` to write the subet to an Excel file for further work on it there or in JMP software

In [18]:
mask = dfSales['Country'] == 'USA'
dfSales_USA = dfSales[mask]
dfSales_USA.index.size

3074

In [19]:
dfSales_USA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3074 entries, 0 to 24545
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Quarter                3074 non-null   object
 1   Channel                3074 non-null   object
 2   Customer ID            3074 non-null   object
 3   Country                3074 non-null   object
 4   Product Line           3074 non-null   object
 5   Revenue                3074 non-null   int64 
 6   Product Cost           3074 non-null   int64 
 7   Customer Service Cost  3074 non-null   int64 
 8   Profit                 3074 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 240.2+ KB


In [20]:
dfSales_USA.describe()

,Revenue,Product Cost,Customer Service Cost,Profit
count,3074.000000,3074.000000,3074.000000,3074.00000
mean,622.608979,519.256994,90.437215,12.89948
std,1051.273629,953.013442,172.169020,432.63212
min,1.000000,0.000000,0.000000,-4139.00000
25%,59.000000,40.000000,7.000000,-42.00000
50%,209.000000,157.500000,27.000000,0.00000
75%,669.500000,530.000000,89.000000,52.00000
max,7540.000000,9256.000000,1865.000000,3664.00000


### Accessing Specific DataFrame Rows
This gives additional examples with the two methods for specifying particular row and column ranges.  The first is the `.loc[]` command that allows specifying by by index values (may be text or numbers) and by column names.  The other is `.loc[]` that allows specifying by row and/or column number. </br></br>
Here is an example of referring to (and printing out) just one row using the row's index and the `.loc[]` command.</br></br>In the example, Row 0 prints out with a list of its values by column, its index name ('0' in this case) and its data type. However, the "object" resulting from the command is a Pandas "Series" that is like an indexed vector.  It can also be written to a file.  The "object" data type is the "O-blood type" type object used when strings/non-numerics are involved.

In [ ]:
First, we need to reset `df_little` index back to a Range index

In [22]:
df_little = df_little.reset_index()
df_little

,one_column,second_column,third_column
0,a,10,100
1,b,15,150
2,c,20,200


In [23]:
df_little.loc[0]

one_column         a
second_column     10
third_column     100
Name: 0, dtype: object

</br></br> This further example uses a `.loc[rowindex, column]` notation to pull out a specific "cell" value:

In [25]:
df_little.loc[1, 'second_column']

15

If you are not familiar with Pandas, the index may just look like an Excel row number that starts at zero instead of one.  While that is true for Pandas' default index, the index can be anything and does not have to be an integer.  There can even be multiple indices to make it easy to (in another, hypothetical DataFrame) call up the data from "Ohio" (first index) "Hamilton County" (second index) using something like: `df.loc[['Ohio','Hamilton']]` instead of just an integer. </br></br></br>Here is an example of reassigning our DataFrame's index to be Fiscal Years

In [26]:
df_little.index = ['FY19/20', 'FY20/21', 'FY21/22']
df_little

,one_column,second_column,third_column
FY19/20,a,10,100
FY20/21,b,15,150
FY21/22,c,20,200


So now the statement below gives us the 20/21 second column value.</br></br>The line after that shows that there is even still a way to just say, "give me the third row, second column value."  That is using the `iloc` command (aka integer location) instead of `loc' </br></br>**Finally:** Don't forget that, in Python (and therefore Pandas), indices always start at zero, so [2,1] refers to the third row, second column of our little DataFrame

In [28]:
df_little.loc['FY20/21', 'second_column']

15

In [29]:
df_little.iloc[2,1]

20